In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkFiles
import os
import pandas as pd


In [2]:
""" RESOLVE SPARK CONTEXT ERROR
export PYTHONPATH=$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9-src.zip:$PYTHONPATH
export PATH=$SPARK_HOME/bin:$SPARK_HOME/python:$PATH
"""


"""----------------------------------------------------------------------------
CREATE SPARK CONTEXT
CREATE SQL CONTEXT
----------------------------------------------------------------------------"""
sc =SparkContext()
sqlContext = SQLContext(sc)

2022-02-19 20:19:48,602 WARN util.Utils: Your hostname, HPCAI10 resolves to a loopback address: 127.0.1.1; using 192.168.1.85 instead (on interface wifi0)
2022-02-19 20:19:48,604 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
"""df1 = sqlContext.range(2, 10000000, 2)
df2 = sqlContext.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")
step4.collect() # 2500000000000"""
print()

In [4]:
data_dir=""
file = os.path.join(data_dir,"iris.csv")
if os.path.isfile(file):
    panda_df = pd.read_csv(file)

In [5]:
iris_df=sqlContext.createDataFrame(panda_df)
iris_df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)



In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
stringIndexer = StringIndexer(inputCol="variety", outputCol="ind_variety")
"""Error raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.9 than that in driver 3.8, 
PySpark cannot run with different minor versions"""
# créer environnement python 3.9
si_model = stringIndexer.fit(iris_df) 

In [8]:
irisNormDf = si_model.transform(iris_df)
irisNormDf.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)
 |-- ind_variety: double (nullable = false)



In [9]:

irisNormDf.select("variety","ind_variety").distinct().collect()

[Row(variety='Virginica', ind_variety=2.0),
 Row(variety='Versicolor', ind_variety=1.0),
 Row(variety='Setosa', ind_variety=0.0)]

In [10]:
"""--------------------------------------------------------------------------
Perform Data Analytics
-------------------------------------------------------------------------"""

#See standard parameters
irisNormDf.describe().show()

+-------+------------------+------------------+------------------+------------------+---------+------------------+
|summary|      sepal_length|       sepal_width|      petal_length|       petal_width|  variety|       ind_variety|
+-------+------------------+------------------+------------------+------------------+---------+------------------+
|  count|               150|               150|               150|               150|      150|               150|
|   mean| 5.843333333333334|3.0573333333333332|3.7580000000000005|1.1993333333333331|     null|               1.0|
| stddev|0.8280661279778632|0.4358662849366984| 1.765298233259466|0.7622376689603464|     null|0.8192319205190405|
|    min|               4.3|               2.0|               1.0|               0.1|   Setosa|               0.0|
|    max|               7.9|               4.4|               6.9|               2.5|Virginica|               2.0|
+-------+------------------+------------------+------------------+--------------

In [11]:
from pyspark.ml.linalg import Vectors
def transformToLabeledPoint(row) :
    lp = ( row["variety"], row["ind_variety"], \
                Vectors.dense([row["sepal_length"],\
                        row["sepal_width"], \
                        row["petal_length"], \
                        row["petal_width"]]))
    return lp

In [12]:
irisLp = irisNormDf.rdd.map(transformToLabeledPoint)
irisLpDf = sqlContext.createDataFrame(irisLp,["species","label", "features"])

In [13]:
irisLpDf.select("species","label","features").show(10)
irisLpDf.cache()

+-------+-----+-----------------+
|species|label|         features|
+-------+-----+-----------------+
| Setosa|  0.0|[5.1,3.5,1.4,0.2]|
| Setosa|  0.0|[4.9,3.0,1.4,0.2]|
| Setosa|  0.0|[4.7,3.2,1.3,0.2]|
| Setosa|  0.0|[4.6,3.1,1.5,0.2]|
| Setosa|  0.0|[5.0,3.6,1.4,0.2]|
| Setosa|  0.0|[5.4,3.9,1.7,0.4]|
| Setosa|  0.0|[4.6,3.4,1.4,0.3]|
| Setosa|  0.0|[5.0,3.4,1.5,0.2]|
| Setosa|  0.0|[4.4,2.9,1.4,0.2]|
| Setosa|  0.0|[4.9,3.1,1.5,0.1]|
+-------+-----+-----------------+
only showing top 10 rows



DataFrame[species: string, label: double, features: vector]

In [41]:
"""--------------------------------------------------------------------------
Perform Machine Learning
-------------------------------------------------------------------------"""

#Split into training and testing data
(trainingData, testData) = irisLpDf.randomSplit([0.8, 0.2], seed=42)
trainingData.count()
testData.count()
testData.collect()

[Row(species='Setosa', label=0.0, features=DenseVector([4.6, 3.1, 1.5, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([4.8, 3.4, 1.6, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([4.9, 3.1, 1.5, 0.1])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.4, 3.7, 1.5, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([4.6, 3.6, 1.0, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.0, 3.0, 1.6, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.0, 3.2, 1.2, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.4, 3.4, 1.5, 0.4])),
 Row(species='Setosa', label=0.0, features=DenseVector([4.4, 3.2, 1.3, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.0, 3.5, 1.3, 0.3])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.1, 3.4, 1.5, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVector([5.1, 3.8, 1.6, 0.2])),
 Row(species='Setosa', label=0.0, features=DenseVect

In [42]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Create the model
rfClassifier = RandomForestClassifier(maxDepth=3, labelCol="label",\
                featuresCol="features")
dtClassifier = DecisionTreeClassifier(maxDepth=4, labelCol="label",\
                featuresCol="features")
rfModel = rfClassifier.fit(trainingData)
dtModel = dtClassifier.fit(trainingData)

In [43]:
print(f"Decision tree : {dtModel.numNodes, dtModel.depth}")
print(f"RandomForest : {rfModel.getNumTrees, rfModel.getImpurity()}")

Decision tree : (13, 4)
RandomForest : (20, 'gini')


In [44]:
#Predict on the test data
predictions = dtModel.transform(testData)
predictions2 = rfModel.transform(testData)
#predictions.select("prediction","species","label").collect()
#predictions.select("prediction","species","label").collect()

In [45]:
#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
print(f"Decision tree accuracy : {evaluator.evaluate(predictions)}") 
print(f"Random forest accuracy : {evaluator.evaluate(predictions2)}") 

Decision tree accuracy : 0.9705882352941176
Random forest accuracy : 1.0


In [46]:
#Draw a confusion matrix
predictions.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|    7|
|  2.0|       2.0|   13|
|  2.0|       1.0|    1|
|  0.0|       0.0|   13|
+-----+----------+-----+



In [47]:
from pyspark.ml.classification import OneVsRest

gbClassifier = GBTClassifier(maxDepth=3, labelCol="label",\
                featuresCol="features")
ovr = OneVsRest(classifier=gbClassifier)
model = ovr.fit(trainingData)

In [48]:
predictions_ovr = model.transform(testData)
evaluator_2 = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"Gradient Boosted tree accuracy : {evaluator_2.evaluate(predictions_ovr)}") 

Gradient Boosted tree accuracy : 1.0


### Accuracy de chaque modèle

In [49]:
print(f"Decision tree accuracy : {evaluator.evaluate(predictions)}") 
print(f"Random forest accuracy : {evaluator.evaluate(predictions2)}") 
print(f"Gradient Boosted tree accuracy : {evaluator_2.evaluate(predictions_ovr)}") 

Decision tree accuracy : 0.9705882352941176
Random forest accuracy : 1.0
Gradient Boosted tree accuracy : 1.0
